In [1]:
using QuantumOptics
basis = NLevelBasis(2)
using LinearAlgebra

using DynamicPolynomials

using DifferentialEquations

using HDF5

using Dates

using Statistics

include("../LiPoSID.jl")

┌ Warning: Package LinearSolve does not have KrylovKit in its dependencies:
│ - If you have LinearSolve checked out for development and have
│   added KrylovKit as a dependency but haven't updated your primary
│   environment's manifest file, try `Pkg.resolve()`.
│ - Otherwise you may need to report an issue with LinearSolve
│ Loading KrylovKit into LinearSolve from project dependency, future warnings for LinearSolve are suppressed.
└ @ nothing nothing:984


Main.LiPoSID

In [2]:
function read_timeevolution(file_name, state, γ)
    h5open(file_name, "r") do file
        ρᵧ = read(file[state][string(γ)])
        t = ρᵧ["t"]
        ρ₀₀ = ρᵧ["p0"]; Re_ρ₀₁ = ρᵧ["s_re"];  Im_ρ₀₁ = ρᵧ["s_im"]
        ρ_series = []
        t_series = []

        for i in 1:length(t)
            ρᵢ= [ ρ₀₀[i]                      Re_ρ₀₁[i] + im * Im_ρ₀₁[i]
                  Re_ρ₀₁[i] - im * Im_ρ₀₁[i]  1 - ρ₀₀[i]                 ]
            push!(ρ_series, convert(Matrix{ComplexF64}, ρᵢ))
            push!(t_series, convert(Float64, t[i]))
        end
        return(t_series, ρ_series)
    end
end

function kossak_time_evolution(ρₒ, tₒ, tₘₐₓ, dt, H0, C, Fᴼᴺᴮ)
    
    # Precompute the dissipation terms
    function D!(dρ, ρ, Fᴼᴺᴮ, C)
        D = similar(ρ)
        D .= 0
        for (k, fᵢ) in enumerate(Fᴼᴺᴮ)
            for (l, fⱼ) in enumerate(Fᴼᴺᴮ)
                D .+= C[k, l] * (2 * fᵢ * ρ * fⱼ' - ρ * (fⱼ' * fᵢ) - (fⱼ' * fᵢ) * ρ)
            end
        end
        dρ .= D ./ 2
        return dρ
    end

    # Define the differential equation
    function matrix_ode!(dρ, ρ, p, t)
        dρ .= -im * (H0 * ρ - ρ * H0)
        D!(dρ, ρ, Fᴼᴺᴮ, C)
    end

    # Time span
    tspan = (tₒ, tₘₐₓ)

    # Create an ODE problem
    prob = ODEProblem(matrix_ode!, ρₒ, tspan)

    # Solve the ODE using a suitable solver, e.g., Tsit5 for non-stiff problems
    sol = solve(prob, Tsit5(), dt=dt, adaptive=false)

    # Access solution
    ρ_solution = sol.u
    t_solution = sol.t
    
    return t_solution, ρ_solution
end

kossak_time_evolution (generic function with 1 method)

For the two level system we can use Kossakowski form:

Kossakowski, Bull. Acad. Pol. Sci. Ser. Math. Astr. Phys. 21, 649 (1973).

Completely positive dynamical semigroups of Nlevel systems
Vittorio Gorini, Andrzej Kossakowski, and E. C. G. Sudarshan
Citation: J. Math. Phys. 17, 821 (1976); doi: 10.1063/1.522979


A linear operator $\frac{d\rho}{dt} : M(2) \rightarrow M(2)$ is the generator of a dynamical semigroup $t \rightarrow \Lambda_t$ of $M(2)$ iff it can be written in the form:

$\frac{d\rho}{dt} = -i[H,\rho] + \frac{1}{2} \sum_{i,j}^3 C_{ij} \{[f_i, \rho f_j] + [f_i\rho, f_j]\}, \quad \rho \in M(2)$

where $C \succeq 0$ - semidefinite matrix


We can also conside master equation in Lindblad form:

$
    \frac{d\rho}{dt} = -\frac{i}{\hbar} [H,\rho(t)] + \mathcal{D}[\rho(t)] = - \frac{i}{\hbar}[H, \rho]+\sum_{\ell=1}^{s-1}\left[J_\ell \rho J_\ell^\dagger - \frac{1}{2}\left\{ J_\ell^\dagger J_\ell, \rho \right\} \right],
$


Consider simple Lindblad master equation with one dissipators:

$
     \frac{d\rho}{dt} = - \frac{i}{\hbar}[H, \rho]+\gamma_1\left[J_1 \rho J_1^\dagger - \frac{1}{2}\left\{ J_1^\dagger J_1, \rho \right\} \right]
$,

where Hamiltonian is hermitian with and dissipator is of the simple form:

$
J_1 = \begin{pmatrix} 0 & r \\ 0 & 0 
   \end{pmatrix} = 
   \begin{pmatrix} 0 & \sqrt{\gamma_1} \\ 0 & 0 
   \end{pmatrix}
$

If use Pauli matrices $\sigma_x, \sigma_y, \sigma_z$ to choose 3 orthonormal $f_i$ for the two-level system to fulfill:

$f_i f_j = \frac{1}{4}\delta_{ij}I  +\frac{i}{2} \sum_{k=1}^3 \epsilon_{ijk} f_k$ which means:

$ \operatorname{tr}(f_i f_j)=\frac{1}{2}\delta_{ij}, \quad \operatorname{tr}(f_i) = 0$

we can use the halfs of Pauli matices as basis and then

Corrresponding Kossakowski matrix  should be:

$ C_{3\times3} = \begin{pmatrix}
\gamma/4 & -i \gamma /4 & 0 \\
i\gamma/4 & \gamma/4 & 0 \\
0 & 0 & 0
\end{pmatrix}$

In [4]:
γᵢ = "0.25133"

γᶠ = parse(ComplexF64, γᵢ)

@polyvar γ

C = [γ/4     -im*γ/4 0 
     im*γ/4   γ/4    0
     0        0      0]  


Cˢⁱᵈ = subs(C, γ => γᶠ)


@polyvar ϵ 

Hˢⁱᵐ = [ 25.133 0+0im
          0.    -25.133  ] / 2

#Hˢⁱᵈ = subs(Hˢʸᵐᵇ,  ϵ => 25.133) #25.126

#Hˢⁱᵐ = Hˢⁱᵈ

Jˢⁱᵐ  = DenseOperator(basis,[ 0    sqrt(0.25133)
                              0.   0+0im    ])


σˣ = [ 0 1 
       1 0 ]

σʸ = [ 0.   -im*1   #im
       im*1 0    ]

σᶻ = [ 1.  0
       0  -1 ] 

fᴷ₁ = σˣ/2
fᴷ₂ = σʸ/2
fᴷ₃ = σᶻ/2

@assert tr(σˣ/2*σʸ/2) == tr(σˣ/2*σᶻ/2) ==  tr(σʸ/2*σᶻ/2) ≈ 0
@assert tr(σˣ/2*σˣ/2) == tr(σʸ/2*σʸ/2) == tr(σᶻ/2*σᶻ/2) ≈ 1/2

fᴷᴼᴺᴮ = [fᴷ₁, fᴷ₂, fᴷ₃]

3-element Vector{Matrix{ComplexF64}}:
 [0.0 + 0.0im 0.5 + 0.0im; 0.5 + 0.0im 0.0 + 0.0im]
 [0.0 + 0.0im 0.0 - 0.5im; 0.0 + 0.5im 0.0 + 0.0im]
 [0.5 + 0.0im 0.0 + 0.0im; 0.0 + 0.0im -0.5 + 0.0im]

In [5]:
Jˢⁱᵐ

Operator(dim=2x2)
  basis: NLevel(N=2)
 0.0+0.0im  0.501328+0.0im
 0.0+0.0im       0.0+0.0im

In [6]:
DenseOperator(basis, Hˢⁱᵐ)

Operator(dim=2x2)
  basis: NLevel(N=2)
 12.5665+0.0im       0.0+0.0im
     0.0+0.0im  -12.5665+0.0im

In [10]:
evol_data_file_name = "../DATA/ALL_GAMMAS_B4_D10.h5"

dodeca_10_states = ["D"*string(n) for n=1:10];

basis_states = ["B"*string(n) for n=1:4];

train_states = basis_states 
test_states = dodeca_10_states

FminStates = []
FmedianStates = []
FmeanStates = []

for state in test_states # loop over initial states
    
    print(state*" ")

    start_time = time()

    tₛ, ρₛ = read_timeevolution(evol_data_file_name, state, γᵢ)
    ρₛ = convert(Vector{Matrix{ComplexF64}}, ρₛ)
 
    ρᵗˢᵗ = [DenseOperator(basis,Hermitian(ρₜ)) for ρₜ in ρₛ]
    tᵗˢᵗ = convert(Vector{Float64}, tₛ)

    #println("Simulating Lindblad form")
    tˢⁱᵐ, ρˢⁱᵐ  = timeevolution.master(tᵗˢᵗ, ρᵗˢᵗ[1], DenseOperator(basis, Hˢⁱᵐ), [Jˢⁱᵐ])

    ρₒ = ρₛ[1]
    dt = tᵗˢᵗ[2] - tᵗˢᵗ[1]
    tᵉⁿᵈ = tᵗˢᵗ[end]

    #println("Simulating Kossakowski form")
    #tˢⁱᵈ, ρˢⁱᵈ  = kossak_time_evolution(ρₒ, tᵉⁿᵈ, dt, Hˢⁱᵐ, Cˢⁱᵈ, fᴷᴼᴺᴮ)
    tˢⁱᵈ, ρˢⁱᵈ  = kossak_time_evolution(ρₒ, tᵗˢᵗ[1], tᵉⁿᵈ, dt, Hˢⁱᵐ, Cˢⁱᵈ, fᴷᴼᴺᴮ)
    #bˢⁱᵈ = LiPoSID.bloch([ρᵢ.data for ρᵢ in ρˢⁱᵈ])

    F_lme_kos = LiPoSID.fidelity_series(basis, [ρₜ.data for ρₜ in ρˢⁱᵐ], ρˢⁱᵈ)
    #F_sb_kos = LiPoSID.fidelity_series(basis, ρₛ, ρˢⁱᵈ)
    
    #Fˢᵖⁱⁿᵇᴼˢᴼⁿ= LiPoSID.fidelity_series(basis, ρₛ, ρˢⁱᵈ)

    #Fᴸᴹᴱₑₓ = [abs(fidelity(ρ₁, ρ₂)) for (ρ₁, ρ₂) in zip(ρᵗˢᵗ, ρˢⁱᵈ)]   

    #h5open(tests_dir*tests_data_file_name,"cw") do fid
        #γ_group = open_group(fid, γᵢ) # open gamma coupling group
        #init_state_group = create_group(γ_group, state) # create initial state group
        #init_state_group["Fidelity_SB"] = convert.(Float64, Fˢᵖⁱⁿᵇᴼˢᴼⁿ)
        #init_state_group["F"] = convert.(Float64, F)
        #init_state_group["bloch_exact"] = convert.(Float64, bᵗˢᵗ)
        #init_state_group["bloch_sid_lme"] = convert.(Float64, bˢⁱᵈ)
        #init_state_group["bloch_sim_lme"] = convert.(Float64, bˢⁱᵈ)
        #init_state_group["tr_dist_grnd"] = TrDist(ρₛ[1], ρᵍ₀)
        #init_state_group["time"] = tᵗˢᵗ
    #end
    
    FminState = minimum(F_lme_kos)
    FmedianState = median(F_lme_kos)
    FmeanState = mean(F_lme_kos)
    
    push!(FminStates, FminState)
    push!(FmedianStates, FmedianState)
    push!(FmeanStates, FmeanState)

end

# Calculate the mean
F_mean_value = mean(FmeanStates)

# Calculate the median
F_median_value = median(FmedianStates)

# Calculate the min
F_min_value = minimum(FminStates)

println("Median fidelity for "*γᵢ*": ", F_median_value)

D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 Median fidelity for 0.25133: 0.9306909810393986
